<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Felipe Arias


### **Link de repositorio de GitHub:** `https://github.com/FelipeAriasT/MDS7202-.git`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

#Importamos librerias utiles 😸

In [2]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

Requirement already up-to-date: plotly in c:\users\felipe\anaconda3\lib\site-packages (5.4.0)


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [33]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
df_players['National_Position'].value_counts()

Sub    556
LCB     48
GK      47
RCB     46
LB      39
RB      38
RM      34
LM      32
ST      30
LCM     25
RCM     25
LDM     19
CAM     19
RDM     18
RS      18
LS      18
CB       9
CDM      9
CM       9
RW       7
LW       7
LWB      4
RAM      4
LAM      4
RWB      4
RF       3
LF       3
Name: National_Position, dtype: int64

In [5]:
df_players['National_Position'].isnull().values.sum() 


16513

In [6]:
df_players['National_Team']= np.where(df_players['National_Position'].isnull(),"Not Selected", "Select")
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,National_Team
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,Select
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,Select
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,Select
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,Select
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,Select
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,56,14,16,13,13,11,13,15,12,Not Selected
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,53,12,17,12,11,12,13,16,12,Not Selected
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,61,41,44,28,42,35,36,42,37,Not Selected
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,48,15,23,14,12,13,12,24,12,Not Selected


In [7]:
df_players=df_players.dropna(subset=['Club_Position'])

In [8]:
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,National_Team
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,Select
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,Select
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,Select
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,Select
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,Select
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,56,14,16,13,13,11,13,15,12,Not Selected
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,53,12,17,12,11,12,13,16,12,Not Selected
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,61,41,44,28,42,35,36,42,37,Not Selected
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,48,15,23,14,12,13,12,24,12,Not Selected


In [9]:
labels=df_players.loc[:,'National_Team']

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

Se va a realizar una transformación StandarScaler a todas las columnas del dataframe menos a las Height, Sliding_Tackle, Stanfinf_Tackle dado que hay alta correlación con estas variables con otras consideradas y se escoje StandarScaler dado que en la literatura esta transformacion es mejor para problemas de clasificación, el cual es este caso.

In [10]:
col_features=['Weight','Age','Weak_foot','Skill_Moves','Ball_Control','Marking','Aggression','Reactions','Interceptions',
              'Vision','Composure','Crossing','Short_Pass','Long_Pass','Acceleration','Speed','Stamina','Strength',
              'Agility','Jumping','Heading','Shot_Power','Finishing','Long_Shots','Curve','Freekick_Accuracy','Penalties',
              'Volleys','Work_Rate','Preffered_Foot','Nationality','Club_Position']

In [11]:
ct = ColumnTransformer(
    [
        ("Standard",StandardScaler(),
            [
                'Weight',
                'Age',
                'Weak_foot',
                'Skill_Moves',
                'Ball_Control',
                'Marking',
                'Aggression',
                'Reactions',
                'Interceptions',
                'Vision',
                'Composure',
                'Crossing',
                'Short_Pass',
                'Long_Pass',
                'Acceleration',
                'Speed',
                'Stamina',
                'Strength',
                'Agility',
                'Jumping',
                'Heading',
                'Shot_Power',
                'Finishing',
                'Long_Shots',
                'Curve',
                'Freekick_Accuracy',
                'Penalties',
                'Volleys'
            ],
        ),
        
        ("OneHot", OneHotEncoder(sparse=False,handle_unknown ='ignore'), ["Work_Rate", "Preffered_Foot",
                                                                          "Nationality","Club_Position"]),
    ]
)


In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [13]:
tree_pipe = Pipeline(
    [("preprocesamiento", ct), ("tree", DecisionTreeClassifier(criterion="entropy"))]
)

LinearSVC_pipe = Pipeline(
    [("preprocesamiento", ct), ("LinearSVC", LinearSVC(random_state=0, tol=1e-5))]
)

RF_pipe = Pipeline(
    [("preprocesamiento", ct), ("RF", RandomForestClassifier())]
)

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


features=df_players[col_features]
labels = labels


X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels
)

In [15]:
tree_pipe = tree_pipe.fit(X_train, y_train)

In [16]:
y_pred = tree_pipe.predict(X_test)
y_pred

array(['Not Selected', 'Not Selected', 'Not Selected', ...,
       'Not Selected', 'Not Selected', 'Not Selected'], dtype=object)

In [17]:
from sklearn.metrics import confusion_matrix, classification_report
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[5213  236]
 [ 226  129]] 

              precision    recall  f1-score   support

Not Selected       0.96      0.96      0.96      5449
      Select       0.35      0.36      0.36       355

    accuracy                           0.92      5804
   macro avg       0.66      0.66      0.66      5804
weighted avg       0.92      0.92      0.92      5804



In [18]:
LinearSVC_pipe = LinearSVC_pipe.fit(X_train, y_train)

C:\Users\Felipe\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [19]:
y_pred = LinearSVC_pipe.predict(X_test)
y_pred

array(['Not Selected', 'Not Selected', 'Not Selected', ...,
       'Not Selected', 'Not Selected', 'Not Selected'], dtype=object)

In [20]:
from sklearn.metrics import confusion_matrix, classification_report
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[5392   57]
 [ 243  112]] 

              precision    recall  f1-score   support

Not Selected       0.96      0.99      0.97      5449
      Select       0.66      0.32      0.43       355

    accuracy                           0.95      5804
   macro avg       0.81      0.65      0.70      5804
weighted avg       0.94      0.95      0.94      5804



In [21]:
RF_pipe = RF_pipe.fit(X_train, y_train)

In [22]:
y_pred = RF_pipe.predict(X_test)
y_pred

array(['Not Selected', 'Not Selected', 'Not Selected', ...,
       'Not Selected', 'Not Selected', 'Not Selected'], dtype=object)

In [23]:
from sklearn.metrics import confusion_matrix, classification_report
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[5434   15]
 [ 334   21]] 

              precision    recall  f1-score   support

Not Selected       0.94      1.00      0.97      5449
      Select       0.58      0.06      0.11       355

    accuracy                           0.94      5804
   macro avg       0.76      0.53      0.54      5804
weighted avg       0.92      0.94      0.92      5804



**Respuesta**:

```
Como se puede observar el acurracy mide el porcentaje de los casos en el modelo ha acertado lo cual se puede observar que el modelo LinearSVC es el que mejor clasifica a los seleccionados ya que tiene el acurracy mas alto, lo cual es falsa esta metrica para clases desbalanceadas lo cual es este caso. Mientras que para la precisión mide la calidad de los datos, es decir, que porcentaje de jugadores esta realmente seleccionado. En este caso es solo un 68% (en el modelo LinearSVC) de los jugadores que se seleccionaron realmente estaba seleccionado. El Recall corresponde a la cantidad de modelo, es decir, que porcentaje de los clientes que estan seleccionados es capaz de identificar, lo cual en ese ejemplo un 33% de los jugadores son posibles identificar de los que estaban seleccionados realmente.

Mientras que las mejoras planteadas es balancear las clases, agregando la muestra para que las clases de seleccionados y no seleccionados sea parecida.



```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [24]:
ataque = ['ST', 'CF','RF','LF','RS','LS'] 
central_ataque = ['RW', 'CAM', 'LW','LAM','RAM'] 
central = ['RM', 'CM', 'LM','LCM','RCM'] 
central_defensa = ['RWB', 'CDM', 'LWB','LDM','RDM']
defensa = ['RB', 'CB', 'LB','RCB','LCB']
arquero = ['GK']

In [25]:
conditions = [df_players['Club_Position'].isin(ataque),
              df_players['Club_Position'].isin(central_ataque),
              df_players['Club_Position'].isin(central),
              df_players['Club_Position'].isin(central_defensa),
              df_players['Club_Position'].isin(defensa),
              df_players['Club_Position'].isin(arquero)]
 
choices = ['ataque','central_ataque','central','central_defensa','defensa','arquero']
 
df_players['Position'] = np.select(conditions, choices, default=df_players['Club_Position'] )

C:\Users\Felipe\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [26]:
df_players['Position']

0        central_ataque
1        central_ataque
2        central_ataque
3                ataque
4               arquero
              ...      
17583               Sub
17584               Sub
17585               Res
17586               Sub
17587               Sub
Name: Position, Length: 17587, dtype: object

In [27]:
df_players_1=df_players[(df_players['Position']!='Sub')&(df_players['Position']!='Res')]

In [28]:
df_players_1['Position'].value_counts()

defensa            2444
central            1614
ataque              865
central_defensa     741
central_ataque      653
arquero             632
Name: Position, dtype: int64

In [29]:
col_features_1=['Weight','Age','Weak_foot','Skill_Moves','Ball_Control','Marking','Aggression','Reactions','Interceptions',
              'Vision','Composure','Crossing','Short_Pass','Long_Pass','Acceleration','Speed','Stamina','Strength',
              'Agility','Jumping','Heading','Shot_Power','Finishing','Long_Shots','Curve','Freekick_Accuracy','Penalties',
              'Volleys','Work_Rate','Preffered_Foot','Nationality']

In [30]:
ct_1 = ColumnTransformer(
    [
        ("Standard",StandardScaler(),
            [
                'Weight',
                'Age',
                'Weak_foot',
                'Skill_Moves',
                'Ball_Control',
                'Marking',
                'Aggression',
                'Reactions',
                'Interceptions',
                'Vision',
                'Composure',
                'Crossing',
                'Short_Pass',
                'Long_Pass',
                'Acceleration',
                'Speed',
                'Stamina',
                'Strength',
                'Agility',
                'Jumping',
                'Heading',
                'Shot_Power',
                'Finishing',
                'Long_Shots',
                'Curve',
                'Freekick_Accuracy',
                'Penalties',
                'Volleys'
            ],
        ),
        ("OneHot", OneHotEncoder(sparse=False,handle_unknown ='ignore'), ["Work_Rate", "Preffered_Foot","Nationality"]),
    ]
)


In [31]:

LinearSVC_pipe_1 = Pipeline(
    [("preprocesamiento", ct_1), ("LinearSVC", LinearSVC(random_state=0, tol=1e-5))]
)


In [43]:
features_1=df_players_1[col_features_1]
labels_1 = df_players_1.loc[:,'Position']


X_train, X_test, y_train, y_test = train_test_split(
    features_1, labels_1, test_size=0.33, shuffle=True, stratify=labels_1
)

In [44]:
X_train.columns

Index(['Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Marking',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Agility', 'Jumping', 'Heading', 'Shot_Power',
       'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties',
       'Volleys', 'Work_Rate', 'Preffered_Foot', 'Nationality'],
      dtype='object')

In [45]:
LinearSVC_pipe = LinearSVC_pipe_1.fit(X_train, y_train)

C:\Users\Felipe\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [46]:
y_pred = LinearSVC_pipe.predict(X_test)
y_pred

array(['ataque', 'defensa', 'defensa', ..., 'defensa', 'arquero',
       'arquero'], dtype=object)

In [47]:
from sklearn.metrics import confusion_matrix, classification_report
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[208   0   0   0   0   1]
 [  0 241  30  11   1   2]
 [  0  35 337  46  53  62]
 [  0  30 124  51   4   6]
 [  0   0  79   4  74  88]
 [  0   1  29   1  12 764]] 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.79      0.85      0.81       285
        central       0.56      0.63      0.60       533
 central_ataque       0.45      0.24      0.31       215
central_defensa       0.51      0.30      0.38       245
        defensa       0.83      0.95      0.88       807

       accuracy                           0.73      2294
      macro avg       0.69      0.66      0.66      2294
   weighted avg       0.71      0.73      0.71      2294



**Respuesta**:

```
Una mejora que se hizo en base a lo propuesto en el enunciado es que las posiciones que no estaban consideradas, ahora se considerado evitando borrar data y esos datos se asignan de mejor manera. Mientras que el modelo elegido es el mejor de la parte anterior, en donde se tiene un acurracy de 73% de aciertos en las clases correspondiente, para clases no tan desbalanceadas como en la parte anterior.
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La metrica $R^2$ corresponde al porcentaje de variación de la variable de respuesta que explica que su relación con las otras variables que se utilizan para la predicción. Mientras mas $R^2$ mejor es el ajuste del modelo.

Tambien se considera una transformación con MinMax dado que en la literatura esta transformación es recomendable para problemas de regresión.

In [98]:
data=pd.read_csv('salarios.csv')

In [99]:
data=data.merge(df_players, left_on='Player', right_on='Name')

In [100]:
data

,Unnamed: 0,Player,Weekly Salary,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,...,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,National_Team,Position
0,1528,Lionel Messi,1538905.0,Lionel Messi,Argentina,RW,RW,170,72,Left,...,71,85,95,88,89,90,74,85,Select,central_ataque
1,1006,Cristiano Ronaldo,1248536.0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,...,85,92,93,90,81,76,85,88,Select,central_ataque
2,1529,Antoine Griezmann,997032.0,Antoine Griezmann,France,CAM,RS,176,67,Left,...,80,81,88,82,84,75,71,87,Select,ataque
3,2004,Neymar,797726.0,Neymar,Brazil,LW,LW,174,68,Right,...,62,78,89,77,79,84,81,83,Select,central_ataque
4,1530,Gareth Bale,607706.0,Gareth Bale,Wales,RS,RW,183,74,Left,...,86,91,87,90,86,85,76,76,Select,central_ataque
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,986,Michael Langer,903.0,Michael Langer,Austria,NaN,GK,194,92,Right,...,14,15,14,13,12,12,13,11,Not Selected,arquero
1862,2536,Nicolas Lemaître,859.0,Nicolas Lemaître,France,NaN,Res,185,83,Right,...,23,23,17,15,19,14,22,17,Not Selected,Res
1863,1523,Andrea Pinamonti,805.0,Andrea Pinamonti,Italy,NaN,Res,181,72,Right,...,62,57,74,60,43,43,69,43,Not Selected,Res
1864,989,Tim Wiesner,674.0,Tim Wiesner,Germany,NaN,Res,192,87,Right,...,16,19,13,15,18,16,20,14,Not Selected,Res


In [257]:
col_features_2=['Weight','Age','Weak_foot','Skill_Moves','Ball_Control','Marking','Aggression','Reactions',
              'Vision','Composure','Short_Pass','Long_Pass','Acceleration','Speed','Stamina','Strength',
              'Agility','Heading','Shot_Power','Finishing','Long_Shots','Freekick_Accuracy','Penalties',
              'Volleys','Work_Rate']

In [258]:


ct_2 = ColumnTransformer(
    [
        
        ("MinMax",MinMaxScaler(),
            [
                'Weight',
                'Age',
                'Weak_foot',
                'Skill_Moves',
                'Ball_Control',
                'Marking',
                'Aggression',
                'Reactions',
                'Vision',
                'Composure',
                'Short_Pass',
                'Long_Pass',
                'Acceleration',
                'Speed',
                'Stamina',
                'Strength',
                'Agility',
                'Heading',
                'Shot_Power',
                'Finishing',
                'Long_Shots',
                'Freekick_Accuracy',
                'Penalties',
                'Volleys'
            ],
        ),
        
        ("OneHot", OneHotEncoder(sparse=False,handle_unknown ='ignore'), ["Work_Rate"]),
    ]
)




In [259]:
features_2=data[col_features_2]
labels_2 = data.loc[:,'Weekly Salary']


X_train, X_test, y_train, y_test = train_test_split(
    features_2, labels_2, test_size=0.33, shuffle=True)

In [260]:
from sklearn import linear_model


Lasso_pipe = Pipeline(
    [("preprocesamiento", ct_2), ("Lasso", linear_model.Lasso(alpha=0.1))]
)

In [261]:
Lasso_pipe = Lasso_pipe.fit(X_train, y_train)

C:\Users\Felipe\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 60138421862.86426, tolerance: 1017615998.4906604
  positive)


In [262]:
y_pred = Lasso_pipe.predict(X_test)
y_pred

array([  74549.51726341,   46809.74489912,   60204.20662693,
         -5440.83489509,  149228.69616775,   40665.16231869,
         25149.27440278,  165970.09571458,  108975.47488572,
         46518.11153526,   26099.3640221 ,   58422.12677686,
        142595.04228307,   73636.07338122,   67072.85810896,
         65206.1738938 ,   58467.62819267,   62261.09925134,
        104437.33143685,   30990.87966031,  -17014.69867953,
         24558.84846794,   42373.11431289,  -54161.37896137,
         76240.26290581,   -9961.00258723,   32941.33975175,
         61617.95492987,   32476.40300996,  -44297.38493697,
         55075.00017671,  103646.77852329,   18739.65858157,
         67065.45198772,  126819.96450757,   44740.3922956 ,
         70388.77411297,   37076.41830727,   35276.28624771,
         29722.1170449 ,   24663.55010878,    2407.70184835,
         78990.55326182,  150408.25783401,  154315.22632946,
        101511.24498505,   66154.02197254,  -16197.94300295,
        136557.53364711,

In [263]:
print(f'El coeficiente R^2 es: {Lasso_pipe.score(X_test,y_test)}')

El coeficiente R^2 es: 0.30048848938129424


**Respuesta**:


Se pudo obtener una regresión con un $R^2$ de 0.30 que esta fuera del objetivo, un aspecto a mejorar corresponde a utilizar una cantidad menor de variables dentro de las cuales solo dejar las que tienen poca correlación entre ellas, y tambien se podria considerar cantidad de goles del jugaro por partido dado que los sueldos tambien al no ser defensa o mediocampista la presencia o imporantacia de un jugador se ve reflejada en el promedio de goles por partido y mientras mas determinante sea, ganara mejor.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>